In [ ]:
# rewriting from scratch again parts
import torchvision
import torch


class MNIST01Part2:
    def __init__(self):
        pass

    def get_data(self):
        # 60,000 samples, 28x28 pixels / sample
        self.train_dataset = torchvision.datasets.MNIST(train=True, download=True, root='./')
        train_rand = torch.randperm(self.train_dataset.data.shape[0]) # for picking random images/labels, like randint except has no repeats
        self.train_img_tensors = self.train_dataset.data.reshape(-1,784)[train_rand]
        self.Y = self.train_dataset.targets[train_rand]
        
        # 10,000 samples, 28x28 pixels / sample
        self.test_dataset = torchvision.datasets.MNIST(train=False, download=True, root='./')
        test_rand = torch.randperm(self.test_dataset.data.shape[0]) # for picking random images/labels, like randint except has no repeats
        self.test_img_tensors = self.test_dataset.data.reshape(-1,784)[test_rand]
        self.test_Y = self.test_dataset.targets[test_rand]

    def forward(self):
        zero = torch.tensor(0)
        self.batch_size = torch.tensor([32])
        # input nodes batch
        # 32 x 784
        self.X = self.train_img_tensors[:self.batch_size] / 255

        h1_nodes = 128
        w1 = torch.randn((h1_nodes, 784))
        w1 = w1 / h1_nodes**0.5
        l1 = self.X@w1.T
        relu1 = torch.maximum(zero, l1)

        h2_nodes = 64
        w2 = torch.randn((h2_nodes, h1_nodes))
        w2 = w2 / h2_nodes**0.5
        l2 = relu1@w2.T
        relu2 = torch.maximum(zero, l2)

        out_nodes = 10
        w3 = torch.randn((out_nodes, h2_nodes))
        w3 = w3 / out_nodes**0.5
        self.b3 = torch.randn()
        logits = relu2@w3.T
        
        self.counts = torch.e**logits
        self.prob =  self.counts / torch.sum(self.counts, dim=1, keepdim=True) # prob = softmax. softmax is an activation function, as is relu
        return self.prob
    
    def cce_loss(self):
        self.pre_pre_loss = self.prob[self.Y]
        self.nlog = -torch.log(self.pre_pre_loss)
        self.loss_top = torch.sum(self.nlog)
        self.loss = self.loss_top / self.batch_size

        return self.loss
    
b = MNIST01Part2()
# b.get_data()
# b.forward()
# b.cce_loss()
# b.backward()


In [ ]:
# b.get_data()
# batch_size = 32
# X = b.train_img_tensors[:batch_size] / 255
# Y = b.Y
# zero = torch.tensor(0)

In [ ]:
# h1_nodes = 128
# w1 = torch.randn((h1_nodes, 784))
# w1 = w1 / h1_nodes**0.5
# l1 = X@w1.T
# relu1 = torch.maximum(zero, l1)

# out_nodes = 10
# w2 = torch.randn((out_nodes, h1_nodes)) * 0.0001
# logits = relu1@w2.T

# probs = torch.e**logits / torch.sum(torch.e**logits, dim=1, keepdim=True) # probs = softmax

# loss = -torch.log(probs[range(batch_size),Y[:batch_size]]).mean()
# loss


# ----------- grads ----------

In [5]:
# current focus
import torchvision
import torch


# ------------------- Get Data -------------------
# 60,000 samples, 28x28 pixels / sample
train_dataset = torchvision.datasets.MNIST(train=True, download=True, root='./')
train_rand = torch.randperm(train_dataset.data.shape[0]) # for picking random images/labels, like randint except has no repeats
train_img_tensors = train_dataset.data.reshape(-1,784)[train_rand]
Y = train_dataset.targets[train_rand]

# 10,000 samples, 28x28 pixels / sample
test_dataset = torchvision.datasets.MNIST(train=False, download=True, root='./')
test_rand = torch.randperm(test_dataset.data.shape[0]) # for picking random images/labels, like randint except has no repeats
test_img_tensors = test_dataset.data.reshape(-1,784)[test_rand]
test_Y = test_dataset.targets[test_rand]


# ------------------- Train -------------------
zero = torch.tensor(0)
batch_size = torch.tensor([32])
# input nodes batch
# 32 x 784
X = train_img_tensors[:batch_size] / 255

h1_nodes = 128
w1 = torch.randn((h1_nodes, 784))
w1 = w1 / h1_nodes**0.5
l1 = X@w1.T
relu1 = torch.maximum(zero, l1)

h2_nodes = 64
w2 = torch.randn((h2_nodes, h1_nodes))
w2 = w2 / h2_nodes**0.5
l2 = relu1@w2.T
relu2 = torch.maximum(zero, l2)

out_nodes = 10
w3 = torch.randn((out_nodes, h2_nodes))
# w3[:,1] += 10 # manually adjusts weights
w3 = w3 / out_nodes**0.5
logits = relu2@w3.T

e_to_the_logits = torch.e**logits
probs = e_to_the_logits / torch.sum(e_to_the_logits, dim=1, keepdim=True) # prob = softmax. softmax is an activation function, as is relu

# ------------------- Loss -------------------
# loss = -torch.log()


# understand this before moving on to anything else
loss = -torch.log(probs[range(batch_size), Y[:batch_size]]).mean()
loss

tensor(3.0460)

In [31]:
-torch.log(probs[range(batch_size), Y[:batch_size]]).mean()

tensor(3.0269)

In [44]:
probs[0,4] += 0.1
probs[0,1] -= 0.1
probs
-torch.log(probs[range(batch_size), Y[:batch_size]])
(probs[range(batch_size), Y[:batch_size]]/1000 + 0.99).mean()
-torch.log(probs[range(1), Y[:1]])

tensor([-0.1121])

In [38]:
-torch.log(probs[range(1), Y[:1]])

tensor([0.6565])

In [26]:
list(range(1)), Y[:1]

([0], tensor([4]))

In [25]:
-torch.log(probs[0,0])

tensor(1.5825)